# Model Training

Run this notebook to train a basic neural network by using the preprocessed data and the Keras high-level TensorFlow API.

At the bottom of the notebook, after model training, add the code to convert the trained model to ONNX format.

First, import the necessary libraries and deactivate GPU acceleration.

In [1]:
from os import environ

environ['CUDA_VISIBLE_DEVICES'] = '-1'

from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import Adam
from numpy import load
from onnx import save
from tf2onnx import convert

2023-11-08 15:16:28.425790: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-08 15:16:30.218302: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-11-08 15:16:30.218385: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-11-08 15:16:30.218395: W 

Next, create a basic neural network, and train the network by using the preprocessed data.

In [2]:
print('training model')

epoch_count = int(environ.get('epoch_count', '20'))
learning_rate = float(environ.get('learning_rate', '0.001'))

Xsm_train = load(f'data/training_samples.npy')
ysm_train = load(f'data/training_labels.npy')
n_inputs = Xsm_train.shape[1]

model = Sequential([
    Dense(n_inputs, input_shape=(n_inputs, ), activation='relu'),
    Dense(32, activation='relu'),
    Dense(2, activation='softmax'),
])
model.compile(
    Adam(learning_rate=learning_rate),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'],
)
model.fit(
    Xsm_train,
    ysm_train,
    validation_split=0.2,
    batch_size=300,
    epochs=epoch_count,
    shuffle=True,
    verbose=2,
)

training model
Epoch 1/20
1214/1214 - 2s - loss: 0.0726 - accuracy: 0.9718 - val_loss: 0.0263 - val_accuracy: 0.9920 - 2s/epoch - 2ms/step
Epoch 2/20
1214/1214 - 2s - loss: 0.0136 - accuracy: 0.9968 - val_loss: 0.0053 - val_accuracy: 1.0000 - 2s/epoch - 1ms/step
Epoch 3/20
1214/1214 - 2s - loss: 0.0070 - accuracy: 0.9986 - val_loss: 0.0097 - val_accuracy: 0.9995 - 2s/epoch - 1ms/step
Epoch 4/20
1214/1214 - 2s - loss: 0.0046 - accuracy: 0.9991 - val_loss: 0.0019 - val_accuracy: 1.0000 - 2s/epoch - 1ms/step
Epoch 5/20
1214/1214 - 2s - loss: 0.0037 - accuracy: 0.9993 - val_loss: 0.0029 - val_accuracy: 1.0000 - 2s/epoch - 1ms/step
Epoch 6/20
1214/1214 - 2s - loss: 0.0030 - accuracy: 0.9994 - val_loss: 0.0018 - val_accuracy: 0.9997 - 2s/epoch - 1ms/step
Epoch 7/20
1214/1214 - 2s - loss: 0.0025 - accuracy: 0.9995 - val_loss: 0.0027 - val_accuracy: 1.0000 - 2s/epoch - 1ms/step
Epoch 8/20
1214/1214 - 2s - loss: 0.0023 - accuracy: 0.9996 - val_loss: 8.7773e-04 - val_accuracy: 1.0000 - 2s/epoch 

# Export the model to ONNX

In [3]:
onnx_model, _ = convert.from_keras(model)
save(onnx_model, 'model.onnx')